In [ ]:
import math

import tqdm
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt


In [ ]:
# linear congruential generator
def lcg(x_0: int, a: int, c: int, M: int):
    """
    x_0: seed
    a: multiplier
    c: increment
    M: modulus and number of possible samples
    """

    if math.gcd(a, M) != 1:
        raise ValueError('a and M must be coprime')
    assert np.all(isinstance(i, int) for i in [x_0, a, c, M]), 'All inputs must be integers'
    
    x = x_0
    U = np.empty(M)
    for i in range(M):
        x = (a * x + c) % M
        U[i] = x / M
    return U


In [ ]:
k = 127
n = 10_000
a = 1664521
c = 1013904223
u = lcg(0, a=a, c=c, M=n)


fig, ax = plt.subplots(1, 3, figsize=(15, 5))

ax[0].hist(u, bins=k, density=True)
ax[1].plot(np.sort(u), np.linspace(0, 1, len(u), endpoint=False))
ax[1].plot((0, 1), (0, 1), 'k--')
ax[1].ecdf(u)
ax[2].plot(u, '.', markersize=1)

plt.show()


# chi-squared test
expected = np.full(k, n / k)
observed, _ = np.histogram(u, bins=k)
chi_squared = np.sum((observed - expected) ** 2 / expected)
print(f'Chi-squared: {chi_squared:.2f}')


# Kolmogorov-Smirnov test
u_sorted = np.sort(u)
D_plus = np.max(np.arange(1, n + 1) / n - u_sorted)
D_minus = np.max(u_sorted - np.arange(0, n) / n)
D = max(D_plus, D_minus)
print(f'Kolmogorov-Smirnov: {D:.2f}')


# Wald-Wolfowitz runs test
runs = 1
for i in range(1, n):
    if u[i] != u[i - 1]:
        runs += 1

expected_runs = 1 + 2 * np.sum(u) * (1 - np.sum(u))
std_dev = np.sqrt(2 * np.sum(u) * (1 - np.sum(u)) * (2 * np.sum(u) * (1 - np.sum(u)) - n) / n)
z = (runs - expected_runs) / std_dev
print(f'Wald-Wolfowitz runs: {runs}, z-score: {z:.2f}')


In [ ]:
stats.chi2(df=k - 1).ppf(0.5)